# Transfer Learning

## Carregando dados do drive

In [ ]:
from google.colab import drive
drive.mount._DEBUG = False
drive.mount('/content/drive', force_remount=True)

In [ ]:
#Carregue os dados PKLotSegmented.tar.gz para seu Google Drive antes de executar
#Ajuste o path para apontar para o local do arquivo

!mkdir /content/datasets

!tar -xf "/content/drive/MyDrive/PKLotSegmented.tar.gz" -C "/content/datasets"
#Ou, se o .tar.gz já estiver carregado no seu ambiente:
!tar -xf "/content/PKLotSegmented.tar.gz" -C "/content/datasets"

!mv /content/datasets/Treino-UFPR04 /content/datasets/treino
!mv /content/datasets/Validação-UFPR05 /content/datasets/validacao
!mv /content/datasets/Teste-PUC /content/datasets/teste

In [ ]:
import math
import copy

import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

## Dataloaders

In [ ]:
img_dim = 224

data_transforms = {
    "treino": transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "validacao": transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
    "teste": transforms.Compose([
        transforms.Resize((img_dim, img_dim)),
        transforms.ToTensor()
    ]),
}

sets = ["treino", "validacao", "teste"]

data_dir = "/content/datasets"
image_datasets = {x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
                  for x in sets}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=0)
               for x in sets}

dataset_sizes = {x: len(image_datasets[x]) for x in sets}
class_names = image_datasets['treino'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("Conectado em um ambiente com", device)

## Instanciando a rede



In [ ]:
model = models.mobilenet_v3_small(pretrained='MobileNet_V3_Small_Weights.IMAGENET1K_V1')
#Use print(model_ft.classifier) para ver a estrutura original da camada de classificação da rede pré-treinada
#A rede original possui 1024 neurônios fully-connected na última camada convolucional da rede
#Os resultados desses neurônios passam por uma função de ativação hardswish
# Depois, 1000 neurônios dão as classes finais, pois a rede foi treinada para 1000 classes
# Essa linha abaixo troca a última camada de 1000 neurônios de classes finais por 2, já que o problema é binário
model.classifier[-1] = nn.Linear(1024, 2)

#Congelando os pesos de todas as camadas, exceto a de classificação
for name, param in model.named_parameters():
  if "classifier" in name:
    param.requires_grad = True
  else:
    param.requires_grad = False


## Definindo função de treino

In [ ]:
# Implementa Early Stopping e Grace Period
def train_model(model, criterion, optimizer, max_epochs=10, grace_period = 3):
    best_loss = math.inf
    curr_grace_period = 0
    best_model = copy.deepcopy(model.state_dict())

    for epoch in range(max_epochs):
        print(f"Época {epoch+1}/{max_epochs}")
        print('-' * 10)

        for phase in ["treino", "validacao"]:
            if phase == "treino":
                model.train()  # Colocar o modelo em modo de treino
            else:
                model.eval()   # Colocar o modelo em modo de avaliação

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == "treino"):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == "treino":
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            print()

            if phase == "validacao":
              if epoch_loss < best_loss:
                best_loss = epoch_loss
                curr_grace_period = 0
                best_model = copy.deepcopy(model.state_dict())
              else:
                curr_grace_period += 1
                if curr_grace_period >= grace_period:
                  print("Early stopping")
                  model.load_state_dict(best_model)
                  return

    model.load_state_dict(best_model)
    return

## Treinando a rede

In [ ]:
#Treinamento
model = model.to(device)

# Função de loss e o otimizador
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model.parameters(), lr=0.005)

# Treinar o modelo
print("Treinando")
train_model(model, criterion, optimizer_ft, max_epochs=10, grace_period=5)

# Avaliar o modelo no conjunto de teste
print("Avaliando no conjunto de testes")
model.eval()
test_corrects = 0

for inputs, labels in dataloaders['teste']:
    inputs = inputs.to(device)
    labels = labels.to(device)

    outputs = model(inputs)
    _, preds = torch.max(outputs, 1)
    test_corrects += torch.sum(preds == labels.data)

test_acc = test_corrects.double() / dataset_sizes['teste']
print(f'Test Acc: {test_acc:.4f}')